### 如何把一篇文档特征化？
* 第一种方法：文档看做是words list，统计文档中出现词的个数作为该词的权重。  
这里有个假设：word在doc中出现的越多，那这个word越能代表这个文档，那么它的权重应该越大，所以以word在doc中出现的次数来代表该word在doc中的权重     
如许多文档组成的文集中出现了[w1,w2,w3,...]个word, doc1中[w1,w2,w3,...]分别出现了[1,0,3...]次，那么[1,0,3...]即是doc1的特征向量  

* TF-IDF可以看做是对上述方法的改进   
当然，这样并不准确，因为有的word在每个doc中都会出现很多次。为了减少这种误差，引入了TF-IDF   
词频（TF）=某个词在文章中的出现次数  
逆向文件频率 (inverse document frequency, IDF):  
是一个词语普遍重要性的度量。某一特定词语的IDF，可以由总文件数目除以包含该词语之文件的数目，再将得到的商取对数得到。  
IDF = log2 (总文档数/含有这个词的文档数)

TFIDF = TF x IDF 代替词频作为word 的权重

sklearn中计算tfidf的公式：  https://github.com/scikit-learn/scikit-learn/blob/f0ab589f/sklearn/feature_extraction/text.py#L996
tf-idf(d, t) = tf(t) * idf(d, t)  
主要是idf计算的不同：  
注意这里的log是自然对数！   
标准的idf计算公式：idf(d, t) = log [ n / (df(d, t) + 1) ])  
sklearn与之不同：  
If ``smooth_idf=True`` (the default),  
idf(d, t) = log [ (1 + n) / (1 + df(d, t)) ] + 1  

if ``smooth_idf=False``：
idf(d, t) = log [ n / df(d, t) ] + 1 

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
X_test = ['I sed about sed the lack',
          'of any Actually']

In [7]:
count_vec=CountVectorizer()

In [8]:
count_vec.fit(X_test)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [9]:
print ('\nvocabulary list:\n\n',count_vec.vocabulary_)


vocabulary list:

 {'sed': 5, 'about': 0, 'the': 6, 'lack': 3, 'of': 4, 'any': 2, 'actually': 1}


In [11]:
count_vec.transform(X_test)

<2x7 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [12]:
#(index1,index2) count中：index1表示为第几个句子或者文档，index2为所有语料库中的单词组成的词典的序号。
#count为在这个文档中这个单词出现的次数。
print (count_vec.transform(X_test))

  (0, 0)	1
  (0, 3)	1
  (0, 5)	2
  (0, 6)	1
  (1, 1)	1
  (1, 2)	1
  (1, 4)	1


In [13]:
#doc-token矩阵：每一行表示一个文档，每一列表示相应编号的token。值为token在doc中出现的频数。
count_vec.transform(X_test).toarray()

array([[1, 0, 0, 1, 0, 2, 1],
       [0, 1, 1, 0, 1, 0, 0]])

### TF-IDF  
默认的token_pattern='(?u)\\b\\w\\w+\\b'，两个或两个以上的word，即不考虑单个汉字。

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
X_test = ['没有 你 的 地方 都是 他乡 没有',
          '没有 你 的 旅行 都是 流浪',
          '没有 你 的 旅行 都是 流浪']
stopwords=['都是']
#norm:归一化，值为l2时，只把向量的长度=1，即sqrt(x1^2+x^2...)=1,值为l1，即abs(x1)+abs(x2)...=1
tfidf=TfidfVectorizer(stop_words=stopwords)
tfidf.fit(X_test)
tfidf.vocabulary_

{'没有': 3, '地方': 1, '他乡': 0, '旅行': 2, '流浪': 4}

In [24]:
tfidf.transform(X_test)

<2x5 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [38]:
print(tfidf.transform(X_test))

  (0, 3)	0.37131279241563214
  (0, 1)	0.3143436037921839
  (0, 0)	0.3143436037921839
  (1, 4)	0.36015410466295694
  (1, 3)	0.27969179067408617
  (1, 2)	0.36015410466295694
  (2, 4)	0.36015410466295694
  (2, 3)	0.27969179067408617
  (2, 2)	0.36015410466295694


In [20]:
tfidf.transform(X_test).toarray()

array([[0.6316672 , 0.6316672 , 0.        , 0.44943642, 0.        ],
       [0.        , 0.        , 0.6316672 , 0.44943642, 0.6316672 ]])

In [21]:
tfidf.transform(X_test).todense().tolist()

[[0.6316672017376245, 0.6316672017376245, 0.0, 0.4494364165239821, 0.0],
 [0.0, 0.0, 0.6316672017376245, 0.4494364165239821, 0.6316672017376245]]

In [22]:
tfidf.transform(X_test).todense()

matrix([[0.6316672 , 0.6316672 , 0.        , 0.44943642, 0.        ],
        [0.        , 0.        , 0.6316672 , 0.44943642, 0.6316672 ]])